In [1]:
import pandas as pd

In [2]:
df_stops = pd.read_csv("1989-stops-localised-final.csv")  

In [3]:
df_stops.drop(['Column'], axis=1, inplace=True)
df_stops['stop_name'] = df_stops['stop_name'].astype(str).str.strip()
df_stops['location'] = df_stops['location'].astype(str).str.strip()

In [4]:
df_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2533 entries, 0 to 2532
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   stop_name            2533 non-null   object 
 1   type                 2533 non-null   object 
 2   in_lines             2533 non-null   object 
 3   location             2533 non-null   object 
 4   identifier           469 non-null    object 
 5   coordinate location  466 non-null    object 
 6   previous_in_lines    939 non-null    object 
 7   stop_description     0 non-null      float64
dtypes: float64(1), object(7)
memory usage: 158.4+ KB


In [5]:
split_df = pd.concat([df_stops[col].astype(str).str.split(';', expand=True).stack().str.strip() for col in df_stops.columns], axis=1, keys=df_stops.columns)
df_stops = split_df.groupby(level=0).ffill().reset_index(drop=True)

In [6]:
df_stops = df_stops.fillna('').groupby(['stop_name', 'type', 'location']).agg({
    'in_lines': ', '.join,
    'identifier': ', '.join
}).reset_index()

# Reset the index
df_stops.reset_index(inplace=True)
df_stops.rename(columns={'index': 'stop_id'}, inplace=True)

df_stops.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2668 entries, 0 to 2667
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   stop_id     2668 non-null   int64 
 1   stop_name   2668 non-null   object
 2   type        2668 non-null   object
 3   location    2668 non-null   object
 4   in_lines    2668 non-null   object
 5   identifier  2668 non-null   object
dtypes: int64(1), object(5)
memory usage: 125.2+ KB


In [7]:
df_line = pd.read_csv("../initial/line_data_1989.csv")  

In [8]:
# Replace U+00a0 with regular whitespace in the entire DataFrame
df_line.replace('\u00a0', ' ', regex=True, inplace=True)

In [9]:
df_line.head(2)

,line_name,type,east-west,Stops,Frequency,Length (time),Length (km),year,Info
0,1,bus,West,"Moabit, Waldstrasse - Alt-Moabit - Franklinstr...",15,60.0,NaN,1989,NaN
1,10,bus,West,Lüdenscheider Weg - Burscheider Weg - Gartenf...,15,77.0,NaN,1989,NaN


In [10]:
# Assuming df is your DataFrame and 'stop_id' is the column you want to modify
df_stops['stop_id'] = '1989' + df_stops['stop_id'].astype(str)
df_stops.head(2)

,stop_id,stop_name,type,location,in_lines,identifier
0,19890,Aalemannufer,bus,"52.5733104403596, 13.212604386897702",{'97'},nan
1,19891,Achtermannstrasse,bus,"52.57652468032656, 13.418517187666131",{'52'},nan


In [11]:
df_line['line_name'] = df_line['line_name'].astype(str).str.strip()
df_line['Stops'] = df_line['Stops'].astype(str).str.strip()
df_line['year'] = df_line['year'].astype(int)
df_line['Frequency'] = pd.to_numeric(df_line['Frequency'], errors='coerce').fillna(0).astype(int)
df_line['Length (time)'] = pd.to_numeric(df_line['Length (time)'], errors='coerce').fillna(0).astype(int)

In [12]:
# Define a function to remove leftover parentheses from a column
def remove_double_whitespace(text):
    text = text.replace("   ", " ")
    text = text.replace("  ", " ")
    text = text.replace("–", "-")
    text = text.replace(" - ", " - ")
    return text

# Apply the function to the 'line_stops' column
df_line['Stops'] = df_line['Stops'].apply(remove_double_whitespace)
df_line['Stops'] = df_line['Stops'].str.replace(r'\s*–\s*', ' - ', regex=True)

In [13]:
mask = df_line['Stops'].str.contains(" - ").fillna(True)
none_df = df_line[~mask]
none_df

,line_name,type,east-west,Stops,Frequency,Length (time),Length (km),year,Info


In [14]:
# Filter rows where 'line_stops' is not a string
non_string_rows = df_line[~df_line['Stops'].apply(lambda x: isinstance(x, str))]
non_string_rows

,line_name,type,east-west,Stops,Frequency,Length (time),Length (km),year,Info


In [15]:
### Creating related tables

In [16]:
def extract_first_start_stop(string):
    stations = string.split(" - ")
    first_station = stations[0]
    last_station = stations[-1]
    return f"{first_station}<> {last_station}"

In [17]:
def create_line_df(df):
    line_df = pd.DataFrame({
        'line_id': range(1, 1+len(df)),
        'year': df['year'],
        'line_name': df['line_name'],
        'type': df["type"],
        "start_stop": df['Stops'].apply(extract_first_start_stop),
        "Length (time)": df['Length (time)'],
        "east_west": df['east-west'],
        "Frequency": df['Frequency']
    })
    return line_df

In [18]:
line_df = create_line_df(df_line)

In [19]:
line_df.head(2)

,line_id,year,line_name,type,start_stop,Length (time),east_west,Frequency
0,1,1989,1,bus,"Moabit, Waldstrasse<> Zehlendorf, Sachtlebenst...",60,West,15
1,2,1989,10,bus,"Lüdenscheider Weg<> Lichterfelde, Lindenstrasse",77,West,15


In [20]:
# Assuming df is your DataFrame and 'stop_id' is the column you want to modify
line_df['line_id'] = '1989' + line_df['line_id'].astype(str)
line_df.head(2)

,line_id,year,line_name,type,start_stop,Length (time),east_west,Frequency
0,19891,1989,1,bus,"Moabit, Waldstrasse<> Zehlendorf, Sachtlebenst...",60,West,15
1,19892,1989,10,bus,"Lüdenscheider Weg<> Lichterfelde, Lindenstrasse",77,West,15


In [21]:
def create_line_stops_df(df):
    line_stops = df['Stops'].str.split(' - ', expand=True).stack().reset_index(level=1, drop=True).reset_index(name='stop_name')

    line_stops['stop_order'] = line_stops.groupby('index').cumcount()
    #index starts from 0 so it looks like 1 row is missing but this is not true

    # Clean the 'Stop Name' column by removing whitespace and non-breaking spaces
    line_stops['stop_name'] = line_stops['stop_name'].str.replace(u'\xa0', ' ').str.strip()

    # reset index so that it can be used for foreign key

    return line_stops

In [22]:
line_stops_df = create_line_stops_df(df_line)

In [23]:
for index, row in line_stops_df.iterrows():
    line_stops_df.at[index, "index"] = row["index"] + 1
line_stops_df.rename(columns={'index': 'line_id'}, inplace=True)

line_stops_df

,line_id,stop_name,stop_order
0,1,"Moabit, Waldstrasse",0
1,1,Alt-Moabit,1
2,1,Franklinstrasse,2
3,1,Helmholtzstrasse,3
4,1,Guerickestrasse,4
...,...,...,...
4086,198,Alt-Marzahn,8
4087,198,Elisabethstrasse,9
4088,198,S-Bhf. Springpfuhl,10
4089,198,Kröver Strasse,11


In [24]:
# Assuming df is your DataFrame and 'stop_id' is the column you want to modify
line_stops_df['line_id'] = '1989' + line_stops_df['line_id'].astype(str)
line_stops_df.head(2)

,line_id,stop_name,stop_order
0,19891,"Moabit, Waldstrasse",0
1,19891,Alt-Moabit,1


In [25]:
# get a list of unique Line IDs in the line_stops DataFrame
line_ids = line_stops_df['line_id'].unique()

# loop through each unique Line ID
for line_id in line_ids:
    # get the first and last Stop Name for this Line ID
    first_stop_name = line_stops_df.loc[line_stops_df['line_id'] == line_id, 'stop_name'].iloc[0]
    last_stop_name = line_stops_df.loc[line_stops_df['line_id'] == line_id, 'stop_name'].iloc[-1]
    
    # get the corresponding Start-Stop value from line_df for this Line ID
    start_stop = line_df.loc[line_df['line_id'] == line_id, 'start_stop'].iloc[0]
    
    # check if the first and last Stop Names are contained in the Start-Stop value
    if first_stop_name not in start_stop or last_stop_name not in start_stop:
        print(f"Quality control check failed for line_id {line_id}.")

# I checked that this works using the following code:
# line_stops_df.loc[0, "stop_name"] = "test"
# and got the right result: Quality control check failed for line_id 1001.


In [26]:
line_df.head(3)

,line_id,year,line_name,type,start_stop,Length (time),east_west,Frequency
0,19891,1989,1,bus,"Moabit, Waldstrasse<> Zehlendorf, Sachtlebenst...",60,West,15
1,19892,1989,10,bus,"Lüdenscheider Weg<> Lichterfelde, Lindenstrasse",77,West,15
2,19893,1989,10,strassenbahn,Betriebshof Marzahn<> Betriebshof Weißensee,34,Ost,20


In [27]:
def add_type(line_stops, line_df):
    # Assuming line_id is the common column between line_stops and line_df
    merged_df = pd.merge(line_stops, line_df[['line_id', 'type', "line_name"]], on='line_id', how='left')
    
    # Rename the 'type' column from line_df to 'type_from_line_df' to avoid conflicts
    merged_df.rename(columns={'type': 'type'}, inplace=True)
    
    # Drop the 'type_from_line_df' column if it's not needed in the final result
    # merged_df.drop(columns=['type_from_line_df'], inplace=True)
    
    return merged_df

In [28]:
line_stops_df = add_type(line_stops_df, line_df)

In [29]:
def add_fk(line_stops_df, df_stops):
    # Create a new dataframe with the Stop Name and Stop ID columns
    stop_id_df = df_stops[['stop_name', 'stop_id', 'type', 'in_lines']]

    # Merge the line_stops_df and stop_id_df dataframes based on matching stop names and line_name condition
    line_stops_df = line_stops_df.merge(stop_id_df,
                                        left_on=['stop_name', 'type'],
                                        right_on=['stop_name', 'type'],
                                        how='left')

    # Filter the rows based on the condition that line_name is contained in in_lines
    line_stops_df = line_stops_df[line_stops_df.apply(lambda row: str(row['line_name']) in str(row['in_lines']), axis=1)]

    return line_stops_df

# Assuming line_stops_df and df_stops are your dataframes
# Replace 'stop_name', 'stop_id', 'type', and 'in_lines' with the actual column names you have

line_stops_df = add_fk(line_stops_df, df_stops)


In [30]:
# Calculate the difference between consecutive 'stop_order' values
line_stops_df['diff'] = line_stops_df['stop_order'].diff()

# Use the following line to drop rows where the 'diff' column is 0.0
line_stops_df = line_stops_df[line_stops_df['diff'] != 0.0]

# Identify faulty rows where the difference is not 1 digit behind
faulty_rows = line_stops_df[(line_stops_df['diff'] != 1) & (line_stops_df['stop_order'] != 0)]
faulty_rows

,line_id,stop_name,stop_order,type,line_name,stop_id,in_lines,diff


In [31]:
faulty_rows.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   line_id     0 non-null      object 
 1   stop_name   0 non-null      object 
 2   stop_order  0 non-null      int64  
 3   type        0 non-null      object 
 4   line_name   0 non-null      object 
 5   stop_id     0 non-null      object 
 6   in_lines    0 non-null      object 
 7   diff        0 non-null      float64
dtypes: float64(1), int64(1), object(6)
memory usage: 0.0+ bytes


In [32]:
# convert 'Stop ID' column to numeric values, coercing errors to NaN
line_stops_df['stop_id'] = pd.to_numeric(line_stops_df['stop_id'], errors='coerce')

# check if all values in 'Stop ID' column are numeric
if line_stops_df['stop_id'].notnull().all():
    print("All values in 'stop_id' column are numeric")
else:
    print("There are non-numeric values in 'stop_id' column")
    print(line_stops_df[line_stops_df['stop_id'].isnull()])


All values in 'stop_id' column are numeric


In [33]:
line_stops_df.drop(['line_name', "in_lines", "type", "diff"], axis=1, inplace=True)

In [34]:
line_stops_df.to_csv("line_stops_1989-final.csv")
line_df.to_csv("line_df_1989-final.csv")
df_stops.to_csv("stops_df_1989-final.csv")